In [25]:

import pandas as pd
from data.random_data import RandomOHLCV
from data.ohlcv import ServeNewOHLCV


ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = 10_000, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-50, 60),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

ohlcv

serv = ServeNewOHLCV(ohlcv)
# serv.serv_period(date='01-01-2024', start_time='09:30:00', end_time='16:00:00')
# serv.serv_period(days_ago=3, months_ago=0, start_time='08:30:00', end_time='18:00:00')
start_data = serv.serv_range(dayrange=(-2,-1), start_time='08:30:00', end_time='18:00:00')
start_data


,open,high,low,close,volume
date,,,,,
2024-04-12 08:45:00,147.10,147.62,146.93,147.23,500
2024-04-12 09:00:00,146.93,147.27,146.35,146.72,500
2024-04-12 09:15:00,147.25,147.86,147.21,147.79,21722
2024-04-12 09:30:00,148.15,148.59,147.63,147.72,500
2024-04-12 09:45:00,148.16,148.88,147.80,148.60,16858
...,...,...,...,...,...
2024-04-13 17:00:00,156.94,157.55,156.80,157.41,500
2024-04-13 17:15:00,157.27,157.75,156.84,157.53,16690
2024-04-13 17:30:00,158.02,158.20,157.89,158.09,500


In [6]:
serv.next_bar(histBars=1)

,open,high,low,close,volume
date,,,,,
2024-04-12 19:00:00,99.16,99.36,98.96,99.25,500
2024-04-12 19:15:00,98.97,99.03,98.79,99.03,29309


In [2]:
# serv.serv_range(dayrange=(2,8), start_time='08:30:00', end_time='18:00:00')

In [57]:
from frame.frame import Frame
f = Frame('TSLA')
serv.next_bar(histBars=1)
f.load_ohlcv(serv.current_data)
f.setup_chart() 
f.plot(trading_hours=True)

In [3]:
display(serv.start_data.index[0])
display(serv.current_data.index[-1])
display(ohlcv.index[0])
display(ohlcv.index[-1])

Timestamp('2024-04-12 08:30:00')

Timestamp('2024-04-13 18:15:00')

Timestamp('2024-01-01 00:00:00')

Timestamp('2024-04-14 03:45:00')

In [126]:



next_bar = serv.next_bar(end_slice=1)


f.load_ohlcv(next_bar)
f.setup_chart()  
display(next_bar)
display(f.data.tail())
f.plot(trading_hours=True)
# f.plot_refresh()



None

,open,high,low,close,volume
date,,,,,
2024-02-01 17:40:00,113.24,113.50,112.93,112.93,22645
2024-02-01 17:45:00,113.35,113.41,112.83,112.98,500
2024-02-01 17:50:00,113.38,113.69,112.74,113.07,500
2024-02-01 17:55:00,113.32,113.92,113.10,113.73,500
2024-02-01 18:00:00,114.14,114.34,113.64,113.86,26050


In [4]:
next_bar = serv.next_bar(end_slice=1)
display(next_bar)
f = Frame('TSLA')
f.load_ohlcv(next_bar)
f.setup_chart()  
f.chart.refesh()
f.plot()


,open,high,low,close,volume
date,,,,,
2024-02-01 08:40:00,67.21,67.29,67.06,67.2,17363


TypeError: Chart.refesh() missing 1 required positional argument: 'df'

,open,high,low,close,volume
date,,,,,
2024-02-01 09:50:00,150.8,151.18,150.79,151.0,24564


IndexError: list index out of range

In [ ]:

from strategies.ta import MA, MACD

f.add_ta(MA('close', 9),  {'line_style': 'solid', 'colour': 'red', 'thickness': 2}) # creates new column to the dataframe with the moving average eg MA_C9
f.add_ta(MA('close', 21), {'line_style': 'solid', 'colour': 'blue', 'thickness': 2})
f.add_ta(MA('close', 28), {'line_style': 'solid', 'colour': 'green', 'thickness': 2})
# f.add_ta(MACD(fast=('close', 12), slow=('close', 26), signal=('close', 9), faststyle={'colour': 'red'}, slowstyle={'colour': 'blue'}, signalstyle={'colour': 'green', 'thickness': 3})) # creates new columns to the dataframe with the MACD values. eg  looks for the MA_9 and MA_21 columns to calculate the MACD values if not then it will call MA(9) and MA(21) to calculate the MACD values
# f.plot()
display(f.data)
f.ta

,open,high,low,close,volume
date,,,,,
2024-02-01 09:30:00,66.88,66.93,66.70,66.73,28179
2024-02-01 09:35:00,66.58,66.90,66.50,66.71,500
2024-02-01 09:40:00,66.69,66.76,66.47,66.59,10208
2024-02-01 09:45:00,66.46,66.54,66.11,66.25,500
2024-02-01 09:50:00,66.06,66.16,65.87,66.06,23365
...,...,...,...,...,...
2024-02-01 15:40:00,64.65,64.97,64.55,64.86,500
2024-02-01 15:45:00,64.95,65.12,64.84,64.90,21009
2024-02-01 15:50:00,64.99,65.18,64.87,65.05,500


{'MA': (MA(column='close', period=28),
  {'line_style': 'solid', 'colour': 'green', 'thickness': 2},
  'line',
  1)}

In [ ]:
def update_ta_data(self):
    for names, (ta, style, chart_type, row) in self.ta.items():
        print(names, ta, style, chart_type, row)
        new_ta_data = ta.run(self.data)
        print(new_ta_data)
 
    
        self.data[names] = new_ta_data


update_ta_data(f)
f.data

MA MA(column='close', period=28) {'line_style': 'solid', 'colour': 'green', 'thickness': 2} line 1
date
2024-02-01 09:30:00          NaN
2024-02-01 09:35:00          NaN
2024-02-01 09:40:00          NaN
2024-02-01 09:45:00          NaN
2024-02-01 09:50:00          NaN
                         ...    
2024-02-01 15:40:00    65.178929
2024-02-01 15:45:00    65.142500
2024-02-01 15:50:00    65.119286
2024-02-01 15:55:00    65.095000
2024-02-01 16:00:00    65.092857
Name: MA_cl_28, Length: 158, dtype: float64


,open,high,low,close,volume,MA
date,,,,,,
2024-02-01 09:30:00,66.88,66.93,66.70,66.73,28179,NaN
2024-02-01 09:35:00,66.58,66.90,66.50,66.71,500,NaN
2024-02-01 09:40:00,66.69,66.76,66.47,66.59,10208,NaN
2024-02-01 09:45:00,66.46,66.54,66.11,66.25,500,NaN
2024-02-01 09:50:00,66.06,66.16,65.87,66.06,23365,NaN
...,...,...,...,...,...,...
2024-02-01 15:40:00,64.65,64.97,64.55,64.86,500,65.178929
2024-02-01 15:45:00,64.95,65.12,64.84,64.90,21009,65.142500
2024-02-01 15:50:00,64.99,65.18,64.87,65.05,500,65.119286
